In [1]:
import nemo.collections.asr as nemo_asr
import glob
import os
import soundfile as sf
import sounddevice as sd
import tqdm
from torch import nn
from torchmetrics.regression import PearsonCorrCoef
import torch


In [17]:
# !sudo apt-get install libportaudio2

# !pip install sounddevice

In [18]:
def Eng_syl_computing(sentence):
    vowels = ['a', 'e', 'i', 'o', 'u', 'y']
    # unicode_eng_vowels = [1377, 1381, 1383, 1384, 1387, 1400, 1413]
    sentence = sentence.lower()
    syl_count = 0
    for s in sentence:
        if s in vowels:
            syl_count += 1
    return syl_count

In [19]:
asr_model = nemo_asr.models.ASRModel.from_pretrained("stt_en_conformer_transducer_small")

[NeMo I 2024-04-15 11:06:41 cloud:58] Found existing object /home/dianasimonyan/.cache/torch/NeMo/NeMo_2.0.0rc0/stt_en_conformer_transducer_small/a755afe69952642a8410330876938b83/stt_en_conformer_transducer_small.nemo.
[NeMo I 2024-04-15 11:06:41 cloud:64] Re-using file from: /home/dianasimonyan/.cache/torch/NeMo/NeMo_2.0.0rc0/stt_en_conformer_transducer_small/a755afe69952642a8410330876938b83/stt_en_conformer_transducer_small.nemo
[NeMo I 2024-04-15 11:06:41 common:815] Instantiating model from pre-trained checkpoint
[NeMo I 2024-04-15 11:06:42 mixins:172] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-04-15 11:06:43 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/nemo_asr_set_2.0/RES/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 4
    pin_memory: true
    use_start_end_token: true
    trim_silence: false
    max_duration: 20
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data2/nemo_asr_set_2.0/RES/audio__OP_0..4095_CL_.tar
    
[NeMo W 2024-04-15 11:06:43 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /manifests/librispeech/librivox-dev-other.json
    - /manifest

[NeMo I 2024-04-15 11:06:43 features:289] PADDING: 0
[NeMo I 2024-04-15 11:06:43 rnnt_models:222] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0}
[NeMo I 2024-04-15 11:06:43 rnnt_models:222] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0}
[NeMo I 2024-04-15 11:06:43 rnnt_models:222] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0}
[NeMo I 2024-04-15 11:06:43 save_restore_connector:263] Model EncDecRNNTBPEModel was successfully restored from /home/dianasimonyan/.cache/torch/NeMo/NeMo_2.0.0rc0/stt_en_conformer_transducer_small/a755afe69952642a8410330876938b83/stt_en_conformer_transducer_small.nemo.


In [2]:
data_dir = '/home/dianasimonyan/Desktop/Thesis/SpeakingRateEstimation/data/LibriSpeech/test-clean-labeled'
file_pathes = glob.glob(data_dir + "/**/*.wav", recursive=True)

In [3]:
len(file_pathes)

2620

In [22]:
# # Extract data and sampling rate from file
# sd.play(data, fs)
# status = sd.wait()  # Wait until file is done playing

In [23]:
labels_csyl = []
labels_sp_rate = []
preds_csyl = []
preds_sp_rate = []

for file in tqdm.tqdm(file_pathes):
    # print('audio path: ', file)
    
    csyl = int(file.split(os.sep)[-1].split('.')[0].split('_')[-1])
    labels_csyl.append(csyl)
    # print('csyl: ', csyl)

    audio, sr = sf.read(file, dtype='float32')
    # sd.play(audio, sr)
    # status = sd.wait()
    audio_len = audio.shape[0]/sr
    sp_rate = csyl/audio_len
    labels_sp_rate.append(sp_rate)
    # print('sp_rate: ', sp_rate)
    transcript = asr_model.transcribe(file)

    pred_csyl = speaking_rate(transcript[0][0])
    preds_csyl.append(pred_csyl)
    pred_sp_rate = pred_csyl/audio_len
    preds_sp_rate.append(pred_sp_rate)
    # print('pred_csyl: ', pred_csyl)
    # print('pred sp_rate: ', pred_sp_rate)
    

Transcribing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.02it/s]

Transcribing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.41it/s]

Transcribing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.12it/s]

Transcribing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.00it/s]

Transcribing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.19it/s]

Transcribing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15.74it/s]

Transcribing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:

In [26]:
print('labels_csyl\n ', labels_csyl[:20])
print('preds_csyl\n ',preds_csyl[:20])
print()
# print('labels_sp_rate\n', labels_sp_rate)
# print('preds_sp_rate\n' , preds_sp_rate)


labels_csyl
  [88, 56, 122, 31, 52, 53, 57, 69, 30, 38, 26, 31, 30, 71, 100, 43, 38, 84, 108, 76]
preds_csyl
  [88, 56, 122, 31, 53, 51, 57, 69, 30, 38, 26, 31, 30, 71, 96, 43, 38, 84, 110, 76]



In [34]:
MSE = nn.MSELoss()
PCC = PearsonCorrCoef()

labels_csyl = torch.tensor(labels_csyl, dtype=torch.float32)
labels_sp_rate = torch.tensor(labels_sp_rate, dtype=torch.float32)
preds_csyl = torch.tensor(preds_csyl, dtype=torch.float32)
preds_sp_rate = torch.tensor(preds_sp_rate, dtype=torch.float32)



In [41]:
mse_csyl = MSE(preds_csyl, labels_csyl)
mse_sp_rate = MSE(preds_sp_rate, labels_sp_rate)

pcc_csyl = PCC(preds_csyl, labels_csyl)
pcc_sp_rate = PCC(preds_sp_rate, labels_sp_rate)

In [42]:
print(f'mse csyl: {mse_csyl.item():.4f}')
print(f'pcc csyl: {pcc_csyl.item():.4f}')
print()

print(f'mse sp_rate: {mse_sp_rate.item():.4f}')
print(f'pcc sp_rate: {pcc_sp_rate.item():.4f}')


mse csyl: 0.3794
pcc csyl: 0.9997

mse sp_rate: 0.0129
pcc sp_rate: 0.9930


### NVIDIA nemo model to remoive silence in the audio


In [4]:
import nemo
import nemo.collections.asr as nemo_asr
vad_model = nemo_asr.models.EncDecClassificationModel.from_pretrained(model_name="vad_marblenet")

[NeMo I 2024-04-15 13:25:37 cloud:58] Found existing object /home/dianasimonyan/.cache/torch/NeMo/NeMo_2.0.0rc0/vad_marblenet/10477085f32c378938ef41e65dc2e1b3/vad_marblenet.nemo.
[NeMo I 2024-04-15 13:25:37 cloud:64] Re-using file from: /home/dianasimonyan/.cache/torch/NeMo/NeMo_2.0.0rc0/vad_marblenet/10477085f32c378938ef41e65dc2e1b3/vad_marblenet.nemo
[NeMo I 2024-04-15 13:25:37 common:815] Instantiating model from pre-trained checkpoint


[NeMo W 2024-04-15 13:25:37 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    vad_stream: false
    manifest_filepath: /home/fjia/code/manifest64/train.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 128
    num_workers: 20
    shuffle: true
    augmentor:
      shift:
        prob: 0.8
        min_shift_ms: -5.0
        max_shift_ms: 5.0
      white_noise:
        prob: 0.8
        min_level: -90
        max_level: -46
    
[NeMo W 2024-04-15 13:25:37 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    vad_stream: false
    manifest_filepath: /home/fjia/code/manifest64/validation.json
    sample_r

[NeMo I 2024-04-15 13:25:38 save_restore_connector:263] Model EncDecClassificationModel was successfully restored from /home/dianasimonyan/.cache/torch/NeMo/NeMo_2.0.0rc0/vad_marblenet/10477085f32c378938ef41e65dc2e1b3/vad_marblenet.nemo.


In [55]:
# from IPython.display import Audio



# for file in tqdm.tqdm(file_pathes):
#     print(vad_model.predict(file))

In [51]:
audio = '/home/dianasimonyan/Desktop/Thesis/SpeakingRateEstimation/data/LibriSpeech/test-clean-labeled/61/70968/16_23.wav'
Audio(audio)